In [1]:
!pip install midiutil
!pip install pydub numpy
!apt-get install ffmpeg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for midiutil: filename=MIDIUtil-1.2.1-py3-none-any.whl size=54569 sha256=9c97eea1b97a5918aadccfadf8784a0a473ccfc406158fbd615728be14389db7
  Stored in directory: /root/.cache/pip/wheels/6c/42/75/fce10c67f06fe627fad8acd1fd3a004a24e07b0f077761fbbd
Successfully built midiutil
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [2]:
#major

from midiutil import MIDIFile
import os
import subprocess
import numpy as np
from scipy.io import wavfile
from pathlib import Path
from google.colab import drive
import time as time_module  # Renamed to avoid conflict

# Mount Google Drive
drive.mount('/content/drive')

# Define the paths
base_dir = '/content/drive/MyDrive/MusicGenerator'
midi_dir = os.path.join(base_dir, "midi")
wav_dir = os.path.join(base_dir, "wav")

# Create directories if they don't exist
chord_type = "major"
midi_chord_dir = os.path.join(midi_dir, "chords", chord_type)
wav_chord_dir = os.path.join(wav_dir, "chords", chord_type)

# Create necessary directories
Path(midi_chord_dir).mkdir(parents=True, exist_ok=True)
Path(wav_chord_dir).mkdir(parents=True, exist_ok=True)

print(f"Created MIDI directory: {midi_chord_dir}")
print(f"Created WAV directory: {wav_chord_dir}")

# Install required packages if needed
try:
    import scipy
except ImportError:
    print("Installing scipy...")
    subprocess.run(['pip', 'install', 'scipy'], check=True)
    from scipy.io import wavfile

noteArray = ["C", "C#", "D", "Eb", "E", "F", "F#", "G", "Ab", "A", "Bb", "B"]
octaveArray = [1, 2, 3, 4, 5, 6, 7, 8]

# Generate note names with octaves (C-1, C-2, etc.)
nameArray = []
for octave in octaveArray:
    for note in noteArray:
        nameArray.append(f"{note}-{octave}")

# Major triad parameters
offset1 = 4  # 4 semitones from root to third in a major triad
offset2 = 3  # 3 semitones from third to fifth in a major triad

# Function to add background noise to audio while preserving original volume
def add_noise(audio_data, noise_level):
    """
    Add background noise to audio at specified level without reducing original volume
    noise_level: 0 to 5, where 0 is no noise and 5 is maximum noise
    Returns: audio with added noise, maintaining original signal strength
    """
    # Scale noise level (0-5) to amplitude
    # Values higher than 0.5 will create very noticeable noise
    noise_amplitude = (noise_level / 5) * 0.008

    # Convert to float64 for better precision during processing
    audio_float = audio_data.astype(np.float64)

    # Get the maximum value for the audio data type
    max_value = np.iinfo(audio_data.dtype).max

    # Check if audio is stereo (2D array) or mono (1D array)
    is_stereo = len(audio_data.shape) > 1 and audio_data.shape[1] > 1

    # Generate white noise instead of Gaussian noise
    if is_stereo:
        # White noise for stereo (same distribution for both channels)
        noise = np.random.uniform(-max_value * noise_amplitude, max_value * noise_amplitude, audio_data.shape).astype(np.float64)
    else:
        # White noise for mono
        noise = np.random.uniform(-max_value * noise_amplitude, max_value * noise_amplitude, audio_data.shape[0]).astype(np.float64)


    # Calculate mixing ratio based on noise level
    # This ensures the original signal maintains its volume
    signal_weight = 1.0  # Keep original signal at 100%

    # Mix signal and noise with proper weighting
    mixed_audio = (signal_weight * audio_float) + noise

    # Clip to valid range for the data type
    mixed_audio_clipped = np.clip(mixed_audio, np.iinfo(audio_data.dtype).min, np.iinfo(audio_data.dtype).max)

    # Convert back to original data type
    return mixed_audio_clipped.astype(audio_data.dtype)

def createTriad(inversion, base, offset1, offset2):
    if(inversion==0):
        note1 = base
        note2 = base+offset1
        note3 = note2+offset2

    elif(inversion==1):
        note1 = base+offset1
        note2 = note1+offset2
        note3 = base+12

    elif(inversion==2):
        note1 = base+offset1+offset2
        note2 = base+12
        note3 = note2+offset1

    return note1, note2, note3

print("Generating major triad chord MIDI files...")

# Counter for generated files
file_count = 0

# Limit to a subset of octaves to keep the number of generated files manageable
limited_octaves = [3, 4, 5]  # Using octaves 3-5

# Create a list to collect note information
note_list = []

for q, j in zip(nameArray, range(24,109)):
    # Get octave from the note name
    note_parts = q.split('-')
    if len(note_parts) == 2 and note_parts[1].isdigit():
        octave = int(note_parts[1])
        # Only process notes in our limited octave range
        if octave in limited_octaves:
            note_list.append((q, j))

# Process the collected notes
for noteName, base in note_list:
    for h in range(0, 3):  # Loop through all 3 inversions
        inversion = h

        appendInversion = "-"+str(inversion)

        note1, note2, note3 = createTriad(inversion, base, offset1, offset2)
        degrees = [note1, note2, note3]

        track    = 0
        channel  = 0
        time_pos = 0    # In beats (renamed from 'time' to avoid conflict)
        duration = 1    # In beats
        tempo    = 60   # In BPM
        volume   = 100  # 0-127, as per the MIDI standard

        MyMIDI = MIDIFile(1)  # One track
        MyMIDI.addTempo(track, time_pos, tempo)

        # Generate chord (all notes played simultaneously)
        for pitch in degrees:
            MyMIDI.addNote(track, channel, pitch, time_pos, duration, volume)

        midiName = noteName+'-'+chord_type+'-chord'+appendInversion+".mid"
        midi_filename = os.path.join(midi_chord_dir, midiName)

        with open(midi_filename, "wb") as output_file:
            MyMIDI.writeFile(output_file)
            file_count += 1

            # Print progress every 10 files
            if file_count % 10 == 0:
                print(f"Generated {file_count} MIDI files...")

print(f"Completed! Generated {file_count} major triad chord MIDI files.")

# Install timidity if not already installed
try:
    subprocess.run(['which', 'timidity'], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("TiMidity++ is already installed.")
except subprocess.CalledProcessError:
    print("Installing TiMidity++...")
    subprocess.run(['apt-get', 'update'], check=True)
    subprocess.run(['apt-get', 'install', '-y', 'timidity'], check=True)
    print("TiMidity++ installed successfully.")

# Convert MIDI files to WAV with different noise levels
print("\nConverting MIDI files to WAV format with varying noise levels (0-5)...")
wav_count = 0

# Get all MIDI files in the chord directory
midi_files = [f for f in os.listdir(midi_chord_dir) if f.endswith('.mid')]
total_midi_files = len(midi_files)
total_conversions = total_midi_files * 6  # 6 versions of each file (noise levels 0-5)

for idx, midi_file in enumerate(midi_files):
    midi_path = os.path.join(midi_chord_dir, midi_file)
    base_name = Path(midi_file).stem

    print(f"Processing {idx+1}/{total_midi_files}: {midi_file}")

    # First, convert to WAV without noise (temp file)
    temp_wav_path = os.path.join('/tmp', f"{base_name}_temp.wav")

    try:
        # Run timidity with a timeout to prevent hanging
        # Increase the volume of the MIDI files when converting to make them more prominent
        timidity_process = subprocess.run(
            ['timidity', midi_path, '-Ow1', '-s', '16kHz', '-A', '200', '-o', temp_wav_path],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE,
            timeout=10  # Set a 10-second timeout for conversion
        )

        # Check if the temp file was created
        if not os.path.exists(temp_wav_path):
            print(f"Warning: Failed to create WAV file for {midi_file}")
            continue

        # Read the WAV file
        sample_rate, audio_data = wavfile.read(temp_wav_path)

        # Create versions with different noise levels
        for noise_level in range(6):  # 0 to 5
            # Add appropriate level of noise
            if noise_level == 0:
                # No noise for level 0
                noisy_audio = audio_data
            else:
                # Add increasing levels of noise
                noisy_audio = add_noise(audio_data, noise_level)

            # Save to final WAV file with noise level in filename
            wav_file = os.path.join(wav_chord_dir, f"{base_name}_noise{noise_level}.wav")
            wavfile.write(wav_file, sample_rate, noisy_audio)
            wav_count += 1

        # Success message for this file
        print(f"Created 6 variations with different noise levels for {midi_file}")

    except subprocess.TimeoutExpired:
        print(f"Warning: Conversion timeout for {midi_file}")
    except Exception as e:
        print(f"Error processing {midi_file}: {str(e)}")

    # Clean up temp file
    if os.path.exists(temp_wav_path):
        os.remove(temp_wav_path)

    # Add a small delay to prevent overwhelming the system
    time_module.sleep(0.1)  # Using the renamed time module

print(f"\nAll done! Generated {file_count} major triad chord MIDI files and {wav_count} WAV files with varying noise levels.")
print(f"MIDI files are in: {midi_chord_dir}")
print(f"WAV files are in: {wav_chord_dir}")

Mounted at /content/drive
Created MIDI directory: /content/drive/MyDrive/MusicGenerator/midi/chords/major
Created WAV directory: /content/drive/MyDrive/MusicGenerator/wav/chords/major
Generating major triad chord MIDI files...
Generated 10 MIDI files...
Generated 20 MIDI files...
Generated 30 MIDI files...
Generated 40 MIDI files...
Generated 50 MIDI files...
Generated 60 MIDI files...
Generated 70 MIDI files...
Generated 80 MIDI files...
Generated 90 MIDI files...
Generated 100 MIDI files...
Completed! Generated 108 major triad chord MIDI files.
Installing TiMidity++...
TiMidity++ installed successfully.

Converting MIDI files to WAV format with varying noise levels (0-5)...
Processing 1/108: C-3-major-chord-0.mid
Created 6 variations with different noise levels for C-3-major-chord-0.mid
Processing 2/108: C-3-major-chord-1.mid
Created 6 variations with different noise levels for C-3-major-chord-1.mid
Processing 3/108: C-3-major-chord-2.mid
Created 6 variations with different noise lev

In [3]:
#minor

from midiutil import MIDIFile
import os
import subprocess
import numpy as np
from scipy.io import wavfile
from pathlib import Path
from google.colab import drive
import time as time_module  # Renamed to avoid conflict

# Mount Google Drive
drive.mount('/content/drive')

# Define the paths
base_dir = '/content/drive/MyDrive/MusicGenerator'
midi_dir = os.path.join(base_dir, "midi")
wav_dir = os.path.join(base_dir, "wav")

# Create directories if they don't exist
chord_type = "minor"  # Changed from "major" to "minor"
midi_chord_dir = os.path.join(midi_dir, "chords", chord_type)
wav_chord_dir = os.path.join(wav_dir, "chords", chord_type)

# Create necessary directories
Path(midi_chord_dir).mkdir(parents=True, exist_ok=True)
Path(wav_chord_dir).mkdir(parents=True, exist_ok=True)

print(f"Created MIDI directory: {midi_chord_dir}")
print(f"Created WAV directory: {wav_chord_dir}")

# Install required packages if needed
try:
    import scipy
except ImportError:
    print("Installing scipy...")
    subprocess.run(['pip', 'install', 'scipy'], check=True)
    from scipy.io import wavfile

noteArray = ["C", "C#", "D", "Eb", "E", "F", "F#", "G", "Ab", "A", "Bb", "B"]
octaveArray = [1, 2, 3, 4, 5, 6, 7, 8]  # Using the full octave range 1-8

# Generate note names with octaves (C-1, C-2, etc.)
nameArray = []
for octave in octaveArray:
    for note in noteArray:
        nameArray.append(f"{note}-{octave}")

# Minor triad parameters
offset1 = 3  # 3 semitones from root to flat third in a minor triad (changed from 4)
offset2 = 4  # 4 semitones from flat third to fifth in a minor triad (changed from 3)

# Function to add background noise to audio while preserving original volume
def add_noise(audio_data, noise_level):
    """
    Add background noise to audio at specified level without reducing original volume
    noise_level: 0 to 5, where 0 is no noise and 5 is maximum noise
    Returns: audio with added noise, maintaining original signal strength
    """
    # Scale noise level (0-5) to amplitude
    # Values higher than 0.5 will create very noticeable noise
    noise_amplitude = (noise_level / 5) * 0.008

    # Convert to float64 for better precision during processing
    audio_float = audio_data.astype(np.float64)

    # Get the maximum value for the audio data type
    max_value = np.iinfo(audio_data.dtype).max

    # Check if audio is stereo (2D array) or mono (1D array)
    is_stereo = len(audio_data.shape) > 1 and audio_data.shape[1] > 1

    # Generate appropriate noise
    if is_stereo:
        # Handle stereo audio (separate noise for each channel)
        noise = np.random.normal(0, max_value * noise_amplitude, audio_data.shape).astype(np.float64)
    else:
        # Handle mono audio
        noise = np.random.normal(0, max_value * noise_amplitude, audio_data.shape[0]).astype(np.float64)

    # Calculate mixing ratio based on noise level
    # This ensures the original signal maintains its volume
    signal_weight = 1.0  # Keep original signal at 100%

    # Mix signal and noise with proper weighting
    mixed_audio = (signal_weight * audio_float) + noise

    # Clip to valid range for the data type
    mixed_audio_clipped = np.clip(mixed_audio, np.iinfo(audio_data.dtype).min, np.iinfo(audio_data.dtype).max)

    # Convert back to original data type
    return mixed_audio_clipped.astype(audio_data.dtype)

def createTriad(inversion, base, offset1, offset2):
    if(inversion==0):
        note1 = base
        note2 = base+offset1
        note3 = note2+offset2

    elif(inversion==1):
        note1 = base+offset1
        note2 = note1+offset2
        note3 = base+12

    elif(inversion==2):
        note1 = base+offset1+offset2
        note2 = base+12
        note3 = note2+offset1

    return note1, note2, note3

print("Generating minor triad chord MIDI files...")  # Changed from major to minor

# Counter for generated files
file_count = 0

# Process all notes in all octaves without limiting
for q, j in zip(nameArray, range(24,109)):
    noteName = q
    base = j

    for h in range(0, 3):  # Loop through all 3 inversions
        inversion = h
        appendInversion = "-"+str(inversion)

        note1, note2, note3 = createTriad(inversion, base, offset1, offset2)
        degrees = [note1, note2, note3]

        track    = 0
        channel  = 0
        time_pos = 0    # In beats (renamed from 'time' to avoid conflict)
        duration = 1    # In beats
        tempo    = 60   # In BPM
        volume   = 100  # 0-127, as per the MIDI standard

        MyMIDI = MIDIFile(1)  # One track
        MyMIDI.addTempo(track, time_pos, tempo)

        # Generate chord (all notes played simultaneously)
        for pitch in degrees:
            MyMIDI.addNote(track, channel, pitch, time_pos, duration, volume)

        midiName = noteName+'-'+chord_type+'-chord'+appendInversion+".mid"
        midi_filename = os.path.join(midi_chord_dir, midiName)

        with open(midi_filename, "wb") as output_file:
            MyMIDI.writeFile(output_file)
            file_count += 1

            # Print progress every 10 files
            if file_count % 10 == 0:
                print(f"Generated {file_count} MIDI files...")

print(f"Completed! Generated {file_count} minor triad chord MIDI files.")  # Changed from major to minor

# Install timidity if not already installed
try:
    subprocess.run(['which', 'timidity'], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("TiMidity++ is already installed.")
except subprocess.CalledProcessError:
    print("Installing TiMidity++...")
    subprocess.run(['apt-get', 'update'], check=True)
    subprocess.run(['apt-get', 'install', '-y', 'timidity'], check=True)
    print("TiMidity++ installed successfully.")

# Convert MIDI files to WAV with different noise levels
print("\nConverting MIDI files to WAV format with varying noise levels (0-5)...")
wav_count = 0

# Get all MIDI files in the chord directory
midi_files = [f for f in os.listdir(midi_chord_dir) if f.endswith('.mid')]
total_midi_files = len(midi_files)
total_conversions = total_midi_files * 6  # 6 versions of each file (noise levels 0-5)

for idx, midi_file in enumerate(midi_files):
    midi_path = os.path.join(midi_chord_dir, midi_file)
    base_name = Path(midi_file).stem

    print(f"Processing {idx+1}/{total_midi_files}: {midi_file}")

    # First, convert to WAV without noise (temp file)
    temp_wav_path = os.path.join('/tmp', f"{base_name}_temp.wav")

    try:
        # Run timidity with a timeout to prevent hanging
        # Increase the volume of the MIDI files when converting to make them more prominent
        timidity_process = subprocess.run(
            ['timidity', midi_path, '-Ow1', '-s', '16kHz', '-A', '200', '-o', temp_wav_path],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE,
            timeout=10  # Set a 10-second timeout for conversion
        )

        # Check if the temp file was created
        if not os.path.exists(temp_wav_path):
            print(f"Warning: Failed to create WAV file for {midi_file}")
            continue

        # Read the WAV file
        sample_rate, audio_data = wavfile.read(temp_wav_path)

        # Create versions with different noise levels
        for noise_level in range(6):  # 0 to 5
            # Add appropriate level of noise
            if noise_level == 0:
                # No noise for level 0
                noisy_audio = audio_data
            else:
                # Add increasing levels of noise
                noisy_audio = add_noise(audio_data, noise_level)

            # Save to final WAV file with noise level in filename
            wav_file = os.path.join(wav_chord_dir, f"{base_name}_noise{noise_level}.wav")
            wavfile.write(wav_file, sample_rate, noisy_audio)
            wav_count += 1

        # Success message for this file
        print(f"Created 6 variations with different noise levels for {midi_file}")

    except subprocess.TimeoutExpired:
        print(f"Warning: Conversion timeout for {midi_file}")
    except Exception as e:
        print(f"Error processing {midi_file}: {str(e)}")

    # Clean up temp file
    if os.path.exists(temp_wav_path):
        os.remove(temp_wav_path)

    # Add a small delay to prevent overwhelming the system
    time_module.sleep(0.1)  # Using the renamed time module

print(f"\nAll done! Generated {file_count} minor triad chord MIDI files and {wav_count} WAV files with varying noise levels.")  # Changed from major to minor
print(f"MIDI files are in: {midi_chord_dir}")
print(f"WAV files are in: {wav_chord_dir}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Created MIDI directory: /content/drive/MyDrive/MusicGenerator/midi/chords/minor
Created WAV directory: /content/drive/MyDrive/MusicGenerator/wav/chords/minor
Generating minor triad chord MIDI files...
Generated 10 MIDI files...
Generated 20 MIDI files...
Generated 30 MIDI files...
Generated 40 MIDI files...
Generated 50 MIDI files...
Generated 60 MIDI files...
Generated 70 MIDI files...
Generated 80 MIDI files...
Generated 90 MIDI files...
Generated 100 MIDI files...
Generated 110 MIDI files...
Generated 120 MIDI files...
Generated 130 MIDI files...
Generated 140 MIDI files...
Generated 150 MIDI files...
Generated 160 MIDI files...
Generated 170 MIDI files...
Generated 180 MIDI files...
Generated 190 MIDI files...
Generated 200 MIDI files...
Generated 210 MIDI files...
Generated 220 MIDI files...
Generated 230 MIDI files...
Generated 240 MIDI files...
Generat

In [4]:
#diminished

from midiutil import MIDIFile
import os
import subprocess
import numpy as np
from scipy.io import wavfile
from pathlib import Path
from google.colab import drive
import time as time_module  # Renamed to avoid conflict

# Mount Google Drive
drive.mount('/content/drive')

# Define the paths
base_dir = '/content/drive/MyDrive/MusicGenerator'
midi_dir = os.path.join(base_dir, "midi")
wav_dir = os.path.join(base_dir, "wav")

# Create directories if they don't exist
chord_type = "diminished"  # Changed to "diminished"
midi_chord_dir = os.path.join(midi_dir, "chords", chord_type)
wav_chord_dir = os.path.join(wav_dir, "chords", chord_type)

# Create necessary directories
Path(midi_chord_dir).mkdir(parents=True, exist_ok=True)
Path(wav_chord_dir).mkdir(parents=True, exist_ok=True)

print(f"Created MIDI directory: {midi_chord_dir}")
print(f"Created WAV directory: {wav_chord_dir}")

# Install required packages if needed
try:
    import scipy
except ImportError:
    print("Installing scipy...")
    subprocess.run(['pip', 'install', 'scipy'], check=True)
    from scipy.io import wavfile

noteArray = ["C", "C#", "D", "Eb", "E", "F", "F#", "G", "Ab", "A", "Bb", "B"]
octaveArray = [1, 2, 3, 4, 5, 6, 7, 8]  # Using the full octave range 1-8

# Generate note names with octaves (C-1, C-2, etc.)
nameArray = []
for octave in octaveArray:
    for note in noteArray:
        nameArray.append(f"{note}-{octave}")

# Diminished triad parameters
offset1 = 3  # 3 semitones from root to minor third
offset2 = 3  # 3 semitones from minor third to diminished fifth (changed from 4)

# Function to add background noise to audio while preserving original volume
def add_noise(audio_data, noise_level):
    """
    Add background noise to audio at specified level without reducing original volume
    noise_level: 0 to 5, where 0 is no noise and 5 is maximum noise
    Returns: audio with added noise, maintaining original signal strength
    """
    # Scale noise level (0-5) to amplitude
    # Values higher than 0.5 will create very noticeable noise
    noise_amplitude = (noise_level / 5) * 0.008

    # Convert to float64 for better precision during processing
    audio_float = audio_data.astype(np.float64)

    # Get the maximum value for the audio data type
    max_value = np.iinfo(audio_data.dtype).max

    # Check if audio is stereo (2D array) or mono (1D array)
    is_stereo = len(audio_data.shape) > 1 and audio_data.shape[1] > 1

    # Generate appropriate noise
    # Generate white noise instead of Gaussian noise
    if is_stereo:
        # White noise for stereo (same distribution for both channels)
        noise = np.random.uniform(-max_value * noise_amplitude, max_value * noise_amplitude, audio_data.shape).astype(np.float64)
    else:
        # White noise for mono
        noise = np.random.uniform(-max_value * noise_amplitude, max_value * noise_amplitude, audio_data.shape[0]).astype(np.float64)


    # Calculate mixing ratio based on noise level
    # This ensures the original signal maintains its volume
    signal_weight = 1.0  # Keep original signal at 100%

    # Mix signal and noise with proper weighting
    mixed_audio = (signal_weight * audio_float) + noise

    # Clip to valid range for the data type
    mixed_audio_clipped = np.clip(mixed_audio, np.iinfo(audio_data.dtype).min, np.iinfo(audio_data.dtype).max)

    # Convert back to original data type
    return mixed_audio_clipped.astype(audio_data.dtype)

def createTriad(inversion, base, offset1, offset2):
    if(inversion==0):
        note1 = base
        note2 = base+offset1
        note3 = note2+offset2

    elif(inversion==1):
        note1 = base+offset1
        note2 = note1+offset2
        note3 = base+12

    elif(inversion==2):
        note1 = base+offset1+offset2
        note2 = base+12
        note3 = note2+offset1

    return note1, note2, note3

print("Generating diminished triad chord MIDI files...")  # Changed to diminished

# Counter for generated files
file_count = 0

# Process all notes in all octaves without limiting
for q, j in zip(nameArray, range(24,109)):
    noteName = q
    base = j

    for h in range(0, 3):  # Loop through all 3 inversions
        inversion = h
        appendInversion = "-"+str(inversion)

        note1, note2, note3 = createTriad(inversion, base, offset1, offset2)
        degrees = [note1, note2, note3]

        track    = 0
        channel  = 0
        time_pos = 0    # In beats (renamed from 'time' to avoid conflict)
        duration = 1    # In beats
        tempo    = 60   # In BPM
        volume   = 100  # 0-127, as per the MIDI standard

        MyMIDI = MIDIFile(1)  # One track
        MyMIDI.addTempo(track, time_pos, tempo)

        # Generate chord (all notes played simultaneously)
        for pitch in degrees:
            MyMIDI.addNote(track, channel, pitch, time_pos, duration, volume)

        midiName = noteName+'-'+chord_type+'-chord'+appendInversion+".mid"
        midi_filename = os.path.join(midi_chord_dir, midiName)

        with open(midi_filename, "wb") as output_file:
            MyMIDI.writeFile(output_file)
            file_count += 1

            # Print progress every 10 files
            if file_count % 10 == 0:
                print(f"Generated {file_count} MIDI files...")

print(f"Completed! Generated {file_count} diminished triad chord MIDI files.")  # Changed to diminished

# Install timidity if not already installed
try:
    subprocess.run(['which', 'timidity'], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("TiMidity++ is already installed.")
except subprocess.CalledProcessError:
    print("Installing TiMidity++...")
    subprocess.run(['apt-get', 'update'], check=True)
    subprocess.run(['apt-get', 'install', '-y', 'timidity'], check=True)
    print("TiMidity++ installed successfully.")

# Convert MIDI files to WAV with different noise levels
print("\nConverting MIDI files to WAV format with varying noise levels (0-5)...")
wav_count = 0

# Get all MIDI files in the chord directory
midi_files = [f for f in os.listdir(midi_chord_dir) if f.endswith('.mid')]
total_midi_files = len(midi_files)
total_conversions = total_midi_files * 6  # 6 versions of each file (noise levels 0-5)

for idx, midi_file in enumerate(midi_files):
    midi_path = os.path.join(midi_chord_dir, midi_file)
    base_name = Path(midi_file).stem

    print(f"Processing {idx+1}/{total_midi_files}: {midi_file}")

    # First, convert to WAV without noise (temp file)
    temp_wav_path = os.path.join('/tmp', f"{base_name}_temp.wav")

    try:
        # Run timidity with a timeout to prevent hanging
        # Increase the volume of the MIDI files when converting to make them more prominent
        timidity_process = subprocess.run(
            ['timidity', midi_path, '-Ow1', '-s', '16kHz', '-A', '200', '-o', temp_wav_path],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE,
            timeout=10  # Set a 10-second timeout for conversion
        )

        # Check if the temp file was created
        if not os.path.exists(temp_wav_path):
            print(f"Warning: Failed to create WAV file for {midi_file}")
            continue

        # Read the WAV file
        sample_rate, audio_data = wavfile.read(temp_wav_path)

        # Create versions with different noise levels
        for noise_level in range(6):  # 0 to 5
            # Add appropriate level of noise
            if noise_level == 0:
                # No noise for level 0
                noisy_audio = audio_data
            else:
                # Add increasing levels of noise
                noisy_audio = add_noise(audio_data, noise_level)

            # Save to final WAV file with noise level in filename
            wav_file = os.path.join(wav_chord_dir, f"{base_name}_noise{noise_level}.wav")
            wavfile.write(wav_file, sample_rate, noisy_audio)
            wav_count += 1

        # Success message for this file
        print(f"Created 6 variations with different noise levels for {midi_file}")

    except subprocess.TimeoutExpired:
        print(f"Warning: Conversion timeout for {midi_file}")
    except Exception as e:
        print(f"Error processing {midi_file}: {str(e)}")

    # Clean up temp file
    if os.path.exists(temp_wav_path):
        os.remove(temp_wav_path)

    # Add a small delay to prevent overwhelming the system
    time_module.sleep(0.1)  # Using the renamed time module

print(f"\nAll done! Generated {file_count} diminished triad chord MIDI files and {wav_count} WAV files with varying noise levels.")  # Changed to diminished
print(f"MIDI files are in: {midi_chord_dir}")
print(f"WAV files are in: {wav_chord_dir}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Created MIDI directory: /content/drive/MyDrive/MusicGenerator/midi/chords/diminished
Created WAV directory: /content/drive/MyDrive/MusicGenerator/wav/chords/diminished
Generating diminished triad chord MIDI files...
Generated 10 MIDI files...
Generated 20 MIDI files...
Generated 30 MIDI files...
Generated 40 MIDI files...
Generated 50 MIDI files...
Generated 60 MIDI files...
Generated 70 MIDI files...
Generated 80 MIDI files...
Generated 90 MIDI files...
Generated 100 MIDI files...
Generated 110 MIDI files...
Generated 120 MIDI files...
Generated 130 MIDI files...
Generated 140 MIDI files...
Generated 150 MIDI files...
Generated 160 MIDI files...
Generated 170 MIDI files...
Generated 180 MIDI files...
Generated 190 MIDI files...
Generated 200 MIDI files...
Generated 210 MIDI files...
Generated 220 MIDI files...
Generated 230 MIDI files...
Generated 240 MIDI f

KeyboardInterrupt: 

In [ ]:
#augmented

from midiutil import MIDIFile
import os
import subprocess
import numpy as np
from scipy.io import wavfile
from pathlib import Path
from google.colab import drive
import time as time_module  # Renamed to avoid conflict

# Mount Google Drive
drive.mount('/content/drive')

# Define the paths
base_dir = '/content/drive/MyDrive/MusicGenerator'
midi_dir = os.path.join(base_dir, "midi")
wav_dir = os.path.join(base_dir, "wav")

# Create directories if they don't exist
chord_type = "augmented"  # Changed to "augmented"
midi_chord_dir = os.path.join(midi_dir, "chords", chord_type)
wav_chord_dir = os.path.join(wav_dir, "chords", chord_type)

# Create necessary directories
Path(midi_chord_dir).mkdir(parents=True, exist_ok=True)
Path(wav_chord_dir).mkdir(parents=True, exist_ok=True)

print(f"Created MIDI directory: {midi_chord_dir}")
print(f"Created WAV directory: {wav_chord_dir}")

# Install required packages if needed
try:
    import scipy
except ImportError:
    print("Installing scipy...")
    subprocess.run(['pip', 'install', 'scipy'], check=True)
    from scipy.io import wavfile

noteArray = ["C", "C#", "D", "Eb", "E", "F", "F#", "G", "Ab", "A", "Bb", "B"]
octaveArray = [1, 2, 3, 4, 5, 6, 7, 8]  # Using the full octave range 1-8

# Generate note names with octaves (C-1, C-2, etc.)
nameArray = []
for octave in octaveArray:
    for note in noteArray:
        nameArray.append(f"{note}-{octave}")

# Augmented triad parameters
offset1 = 4  # 4 semitones from root to major third
offset2 = 4  # 4 semitones from major third to augmented fifth (changed from 3)

# Function to add background noise to audio while preserving original volume
def add_noise(audio_data, noise_level):
    """
    Add background noise to audio at specified level without reducing original volume
    noise_level: 0 to 5, where 0 is no noise and 5 is maximum noise
    Returns: audio with added noise, maintaining original signal strength
    """
    # Scale noise level (0-5) to amplitude
    # Values higher than 0.5 will create very noticeable noise
    noise_amplitude = (noise_level / 5) * 0.008

    # Convert to float64 for better precision during processing
    audio_float = audio_data.astype(np.float64)

    # Get the maximum value for the audio data type
    max_value = np.iinfo(audio_data.dtype).max

    # Check if audio is stereo (2D array) or mono (1D array)
    is_stereo = len(audio_data.shape) > 1 and audio_data.shape[1] > 1

    # Generate appropriate noise
    # Generate white noise instead of Gaussian noise
    if is_stereo:
        # White noise for stereo (same distribution for both channels)
        noise = np.random.uniform(-max_value * noise_amplitude, max_value * noise_amplitude, audio_data.shape).astype(np.float64)
    else:
        # White noise for mono
        noise = np.random.uniform(-max_value * noise_amplitude, max_value * noise_amplitude, audio_data.shape[0]).astype(np.float64)


    # Calculate mixing ratio based on noise level
    # This ensures the original signal maintains its volume
    signal_weight = 1.0  # Keep original signal at 100%

    # Mix signal and noise with proper weighting
    mixed_audio = (signal_weight * audio_float) + noise

    # Clip to valid range for the data type
    mixed_audio_clipped = np.clip(mixed_audio, np.iinfo(audio_data.dtype).min, np.iinfo(audio_data.dtype).max)

    # Convert back to original data type
    return mixed_audio_clipped.astype(audio_data.dtype)

def createTriad(inversion, base, offset1, offset2):
    if(inversion==0):
        note1 = base
        note2 = base+offset1
        note3 = note2+offset2

    elif(inversion==1):
        note1 = base+offset1
        note2 = note1+offset2
        note3 = base+12

    elif(inversion==2):
        note1 = base+offset1+offset2
        note2 = base+12
        note3 = note2+offset1

    return note1, note2, note3

print("Generating augmented triad chord MIDI files...")  # Changed to augmented

# Counter for generated files
file_count = 0

# Process all notes in all octaves without limiting
for q, j in zip(nameArray, range(24,109)):
    noteName = q
    base = j

    for h in range(0, 3):  # Loop through all 3 inversions
        inversion = h
        appendInversion = "-"+str(inversion)

        note1, note2, note3 = createTriad(inversion, base, offset1, offset2)
        degrees = [note1, note2, note3]

        track    = 0
        channel  = 0
        time_pos = 0    # In beats (renamed from 'time' to avoid conflict)
        duration = 1    # In beats
        tempo    = 60   # In BPM
        volume   = 100  # 0-127, as per the MIDI standard

        MyMIDI = MIDIFile(1)  # One track
        MyMIDI.addTempo(track, time_pos, tempo)

        # Generate chord (all notes played simultaneously)
        for pitch in degrees:
            MyMIDI.addNote(track, channel, pitch, time_pos, duration, volume)

        midiName = noteName+'-'+chord_type+'-chord'+appendInversion+".mid"
        midi_filename = os.path.join(midi_chord_dir, midiName)

        with open(midi_filename, "wb") as output_file:
            MyMIDI.writeFile(output_file)
            file_count += 1

            # Print progress every 10 files
            if file_count % 10 == 0:
                print(f"Generated {file_count} MIDI files...")

print(f"Completed! Generated {file_count} augmented triad chord MIDI files.")  # Changed to augmented

# Install timidity if not already installed
try:
    subprocess.run(['which', 'timidity'], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print("TiMidity++ is already installed.")
except subprocess.CalledProcessError:
    print("Installing TiMidity++...")
    subprocess.run(['apt-get', 'update'], check=True)
    subprocess.run(['apt-get', 'install', '-y', 'timidity'], check=True)
    print("TiMidity++ installed successfully.")

# Convert MIDI files to WAV with different noise levels
print("\nConverting MIDI files to WAV format with varying noise levels (0-5)...")
wav_count = 0

# Get all MIDI files in the chord directory
midi_files = [f for f in os.listdir(midi_chord_dir) if f.endswith('.mid')]
total_midi_files = len(midi_files)
total_conversions = total_midi_files * 6  # 6 versions of each file (noise levels 0-5)

for idx, midi_file in enumerate(midi_files):
    midi_path = os.path.join(midi_chord_dir, midi_file)
    base_name = Path(midi_file).stem

    print(f"Processing {idx+1}/{total_midi_files}: {midi_file}")

    # First, convert to WAV without noise (temp file)
    temp_wav_path = os.path.join('/tmp', f"{base_name}_temp.wav")

    try:
        # Run timidity with a timeout to prevent hanging
        # Increase the volume of the MIDI files when converting to make them more prominent
        timidity_process = subprocess.run(
            ['timidity', midi_path, '-Ow1', '-s', '16kHz', '-A', '200', '-o', temp_wav_path],
            stdout=subprocess.PIPE, stderr=subprocess.PIPE,
            timeout=10  # Set a 10-second timeout for conversion
        )

        # Check if the temp file was created
        if not os.path.exists(temp_wav_path):
            print(f"Warning: Failed to create WAV file for {midi_file}")
            continue

        # Read the WAV file
        sample_rate, audio_data = wavfile.read(temp_wav_path)

        # Create versions with different noise levels
        for noise_level in range(6):  # 0 to 5
            # Add appropriate level of noise
            if noise_level == 0:
                # No noise for level 0
                noisy_audio = audio_data
            else:
                # Add increasing levels of noise
                noisy_audio = add_noise(audio_data, noise_level)

            # Save to final WAV file with noise level in filename
            wav_file = os.path.join(wav_chord_dir, f"{base_name}_noise{noise_level}.wav")
            wavfile.write(wav_file, sample_rate, noisy_audio)
            wav_count += 1

        # Success message for this file
        print(f"Created 6 variations with different noise levels for {midi_file}")

    except subprocess.TimeoutExpired:
        print(f"Warning: Conversion timeout for {midi_file}")
    except Exception as e:
        print(f"Error processing {midi_file}: {str(e)}")

    # Clean up temp file
    if os.path.exists(temp_wav_path):
        os.remove(temp_wav_path)

    # Add a small delay to prevent overwhelming the system
    time_module.sleep(0.1)  # Using the renamed time module

print(f"\nAll done! Generated {file_count} augmented triad chord MIDI files and {wav_count} WAV files with varying noise levels.")  # Changed to augmented
print(f"MIDI files are in: {midi_chord_dir}")
print(f"WAV files are in: {wav_chord_dir}")